# 기본 준비(모듈)

- 모듈 가져오기
- GPU 설정
- 학습 환경에 대한 상수(에포크, 배치사이즈)

In [1]:
# 모듈 가져오기
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# 데이터 관련
from torchvision import transforms, datasets

In [2]:
# GPU 설정
use_cuda = torch.cuda.is_available()
DEVICE   = torch.device( 'cuda' if use_cuda else 'cpu')
# 필요한 텐서에 적용하겟다
DEVICE

device(type='cpu')

In [3]:
# 학습 관련 상수
EPOCHES    = 30 if use_cuda else 5 # gpu 혹은 cpu 상황에 맞춰셔 학습량을 조절하겟다
# 1회 학습시 데이터량
BATCH_SIZE = 64  # 설정값

EPOCHES, BATCH_SIZE

(5, 64)

# 데이터 준비(Fashion)

- 제공되는 데이터를 사용
- API에서 편하게 제공해준다
  - 직접 데이터 만들때 참고!! -> 파이프라인 구축시 참고

In [4]:
# 토치는 데이터로더를 통해서 데이터를 공급한다 - 공급자

# 훈련데이터 공급
train_loader = torch.utils.data.DataLoader( 
    # 데이터셋
    datasets.FashionMNIST(
      root='./data',       # 데이터가 저장된 위치 
      train=True,          # 훈련용 데이터 
      download=True,       # 데이터가 있다면 다운로드 않함
      # 이미지 변환해서(원하는대로, 전처리해서) 제공받을수 있다
      transform = transforms.Compose( [ 
          transforms.ToTensor(),  # 이미지를 텐서로 변환해서 받겟다
          transforms.Normalize( (0.2), (0.3) )  # 이미지 데이터를 정규화 처리 하겟다
      ])
    ),
    # 배치사이즈
    batch_size = BATCH_SIZE,
    # 데이터를 섞을 것인가?
    shuffle    = True
)
# 테스트데이터 공급
test_loader  = torch.utils.data.DataLoader( 
    # 데이터셋
    datasets.FashionMNIST(
      root='./data',       # 데이터가 저장된 위치
      train=False,         # 테스트용 데이터
      download=True,       # 데이터가 있다면 다운로드 않함
      transform = transforms.Compose([
          transforms.ToTensor(),
          transforms.Normalize( (0.2), (0.3) )  # 이미지 데이터를 정규화 처리 하겟다
      ])
    ),
    # 배치사이즈
    batch_size = BATCH_SIZE,
    # 데이터를 섞을 것인가?
    shuffle    = True
)

  0%|          | 0/26421880 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/29515 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/4422102 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



  0%|          | 0/5148 [00:00<?, ?it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


# 신경망 구성

- 이미지 크기가 28, 28
- shape 변화
  - (28,28) : 원본
    - (24,24) : 합성곱층 1차 통과 -> valid
      - (12, 12) : 풀링 통과 -> 크기 2
        - (8, 8) : 합성곱층 2차 통과 -> valid
          - (4, 4) : 풀링 통과 -> 크기 2
- channels 변화
  - 1
    - 10
      - 20
        - 4 * 4 * 20 => 320
          - 50
            - 10

- 합성곱의 커널사이즈
  - 기존과 동일하게 5

In [5]:
# 네트워크 구성 스타일은 객체 지향적 구성
class Net(nn.Module):
  # 생성자
  def __init__(self):
    # 부모 생성자 호출
    super(Net, self).__init__()
    # 맴버 변수 생성 => 요소 생성
    # 커스터마이즈 ===========================    
    self.conv1 = nn.Conv2d( 1, 10,  kernel_size=5, padding='valid')
    self.conv2 = nn.Conv2d( 10, 20, kernel_size=5, padding='valid')
    self.conv2_drop = nn.Dropout2d() # 기본값 그대로 적용 0.5

    # fc1 전단계(펴준다 flattern) (?, 4, 4, 20 ) => 2D( ?, 4*4*20)
    self.fc1   = nn.Linear( 4*4*20, 50)
    # 패션 손글씨의 최종 분류개수는 10개 (0~9)
    self.fc2   = nn.Linear( 50, 10)
    # 커스터마이즈 ===========================
    pass

  # 순전파 신경망 구성
  def forward(self, x):
    # 네트워크 구성 => 연결
    # 커스터마이즈 ===========================
    # 1층
    x = F.relu( F.max_pool2d( self.conv1( x ), 2 ) )

    # 2층
    x = F.relu( F.max_pool2d( self.conv2_drop( self.conv2( x ) ), 2  ) )

    # flattern (4D -> 2D)
    x = x.view( -1, 320)

    # 전결합층 구성    
    x = F.relu( self.fc1(x) ) # 320->50
    # 함수(기능)의 느낌으로 과적합 추가
    # 훈련중일때 적용되게 처리 -> 부모(nn.Module) 객체로 상속받는 기능 중에 
    x = F.dropout( x, training=self.training )
    # 출력층 연결
    x = F.relu( self.fc2(x) ) # 50->10
    
    # 커스터마이즈 ===========================
    return F.log_softmax( x, dim=1 )
    pass
  
  # 오차 역전파 신경망 구성 (생략, 훈련시 적용)
  pass

In [6]:
# 모델 생성 -> 학습전 모델
model = Net().to( DEVICE )

In [7]:
model, model.parameters()

(Net(
   (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1), padding=valid)
   (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1), padding=valid)
   (conv2_drop): Dropout2d(p=0.5, inplace=False)
   (fc1): Linear(in_features=320, out_features=50, bias=True)
   (fc2): Linear(in_features=50, out_features=10, bias=True)
 ), <generator object Module.parameters at 0x7f9b023b4850>)

In [8]:
# 최적화 도구 생성
# model.parameters() : 최적화 대상 => 학습을 통해서 미세조정될 값들
# 경사하강법, W,b를 조정하는데, 값을 점진적으로 기울여가면서 최적화를 수행
# lr : 학습에 대한 비율
# momentum :  관성 -> SGD에 속도를 조정
# 0.01, 0.5 => 잘 만들어진 예시값(설정값)
optimizer = optim.SGD( model.parameters(), lr=0.01, momentum=0.5 )
# 평가도구는 훊련시 적용

# 학습 모듈

In [12]:
def train( model, train_loader, optimizer, epoch ):
  print('한 세대 풀 학습')
  # 학습 모드 전환 
  model.train()

  # 한세대 풀데이터 사용
  # 데이터 추출
  # data:feature, target:label => Tensor
  for batch_idx, (data, target) in enumerate( train_loader ):
    # CPU or GPU 지정
    data   = data.to( DEVICE )
    target = target.to( DEVICE )
    # 최적화 도구의 초기화 (모델의 파라미터는 누적되서 조정) : 누적된값을 제거하기 위해
    optimizer.zero_grad()
    # 모델에 데이터를 주입
    # 순전파 진행 (데이터가 출력까지 흘러 들어간다) y_ 계산된다
    output = model( data )
    # 크로스엔트로피 -> 지표, 손실함수 (예측값, 정답)
    loss   = F.cross_entropy( output, target )
    # (오차)역전파를 통해서 최적화 
    # y -> x 거슬러 올라가서 도달할수 있도록 W,b를 미세조정한다
    loss.backward()
    # 실제 반영. 모델의 파라미터를 갱신한다    
    optimizer.step()
    # 출력
    if batch_idx % 200 == 0:
      # 세대, 이로그가 출력될때까지 사용한 데이량, 손실값
      print( f'Train Epoch: { epoch }, { batch_idx * len(data) } / { len(train_loader) } \tLoss: {loss.item()}' )    


# 테스트 모듈

In [13]:
def test( model, test_loader ):
  print('한 세대 학습 종료후 테스트')
  # 테스트 모드 전환
  model.eval()
  test_loss     = 0 # 손실의 평균값
  test_accuracy = 0 # 정확도의 평균

  # 모델을 이용하여서 예측을 수행하는데 -> 기록하지 않겟다 -> 학습에 영향을 미치지 않겟다
  # 해당 블록의 history를 트래킹 하지 않겟다
  with torch.no_grad():
    # 여기서 작성한 부분인 비공식, 영향 미지치 않는다
    # 아래에서 수행된 모든 손실값, 정확도를 평균내서 로그값을 리턴하겟다
    for data, target in test_loader:
      data, target = data.to(DEVICE), target.to(DEVICE)
      output       = model( data )
      # 오차에 대한 합산
      # item() : 수치로 출력됨
      # 누적합산
      test_loss  += F.cross_entropy( output, target, reduction='sum' ).item()
      # 예측 : 가장 높은 값을 가진 인덱스 -> 0, 1, 2, 3, -> argmax
      # max( .. ) -> [ 최대값, 최대값을가진 인덱스 ]
      pred        = output.max( 1, keepdim=True )[1]
      # 정답과 예측값을 비교하는 형태 => target을 pred 형태로 형변환을 해서 비교
      # 정확도 체크
      test_accuracy += pred.eq( target.view_as( pred ) ).sum().item()
      pass
    pass

  # 평균 손실량 = 누적손실량 / 데이터 세트의 개수
  test_loss     = test_loss / len( test_loader.dataset)

  # 평균 정확도(%)  = 100*누적정확도/데이터 세트의 개수
  test_accuracy = 100. * test_accuracy / len( test_loader.dataset)

  return test_loss, test_accuracy

# 실행(학습->테스트->결과)

In [14]:
# for문 1회가 1세대
for epoch in range( 1, EPOCHES+ 1 ):
  # 훈련 -> 데이터를 풀로 다사용
  train( model, train_loader, optimizer, epoch )

  # 테스트 -> 세대별로 훈련이 끝나면 테스트 진행 => 손실값, 정확도
  test_loss, test_accuracy = test( model, test_loader )

  # 로그
  print( f'[{epoch}] Test Loss: {test_loss:.4f} Accuracy: {test_accuracy:.4f}' )

한 세대 풀 학습
Train Epoch: 1, 0 / 938 	Loss: 2.2952919006347656
Train Epoch: 1, 12800 / 938 	Loss: 1.9567097425460815
Train Epoch: 1, 25600 / 938 	Loss: 1.2174272537231445
Train Epoch: 1, 38400 / 938 	Loss: 0.8871477246284485
Train Epoch: 1, 51200 / 938 	Loss: 0.9645193815231323
한 세대 학습 종료후 테스트
[1] Test Loss: 0.7006 Accuracy: 72.5400
한 세대 풀 학습
Train Epoch: 2, 0 / 938 	Loss: 0.8269128203392029
Train Epoch: 2, 12800 / 938 	Loss: 0.878093957901001
Train Epoch: 2, 25600 / 938 	Loss: 0.6831154823303223
Train Epoch: 2, 38400 / 938 	Loss: 0.7434213161468506
Train Epoch: 2, 51200 / 938 	Loss: 0.6059315204620361
한 세대 학습 종료후 테스트
[2] Test Loss: 0.5972 Accuracy: 76.5100
한 세대 풀 학습
Train Epoch: 3, 0 / 938 	Loss: 0.6263006329536438
Train Epoch: 3, 12800 / 938 	Loss: 1.0494003295898438
Train Epoch: 3, 25600 / 938 	Loss: 0.5518405437469482
Train Epoch: 3, 38400 / 938 	Loss: 0.548417329788208
Train Epoch: 3, 51200 / 938 	Loss: 0.6141288876533508
한 세대 학습 종료후 테스트
[3] Test Loss: 0.5413 Accuracy: 79.2300
한 세대 풀